# Scratchpad for paper revisions

In [1]:
%load_ext autoreload
%autoreload 2
import pickle
import os, sys
root_path = os.path.realpath('../')
sys.path.append(root_path)

import torch
from pathlib import Path
import numpy as np
import random


from utils.data import make_blobs_dataset
from utils.nnet import get_device

from hebbcl.logger import MetricLogger
from hebbcl.model import Nnet
from hebbcl.trainer import Optimiser, train_on_blobs
from hebbcl.parameters import parser
from hebbcl.tuner import HPOTuner

## Hyperparameter optimisation
hpo on network trained with fewer episodes

### HPO: blocked trials with oja_ctx

In [2]:
# HPO on blocked trials with oja_ctx
args = parser.parse_args(args=[])
args.n_episodes = 8
args.hpo_fixedseed = True
args.hpo_scheduler = "bohb"
args.hpo_searcher = "bohb"
# dict(sorted(vars(args).items(),key=lambda k: k[0]))
args.ctx_avg = False
# init tuner
tuner = HPOTuner(args, time_budget=60*15, metric="loss")

tuner.tune(n_samples=500)

df = tuner.results
df = df[["mean_loss", "mean_acc", "config.lrate_sgd","config.lrate_hebb", "config.ctx_scaling","config.seed","done"]]
df = df[df["done"]==True]
df = df.drop(columns=["done"])
df = df.dropna()
df = df.sort_values("mean_loss",ascending=True)

df.reset_index()
print(df.head(15))

print(tuner.best_cfg)

with open("../results/raytune_oja_ctx_blocked_8episodes.pkl", "wb") as f:
    pickle.dump(df, f)

2022-06-23 14:16:47,068	INFO tune.py:636 -- Total run time: 903.65 seconds (900.65 seconds for the tuning loop).


          mean_loss  mean_acc  config.lrate_sgd  config.lrate_hebb  \
trial_id                                                             
16dfc485 -28.530746       1.0          0.092071           0.003988   
087fa24c -28.298903       1.0          0.090435           0.020123   
3cd50ef9 -28.293268       1.0          0.091203           0.003310   
03647346 -28.250048       1.0          0.088752           0.003588   
ad5ea28f -28.114159       1.0          0.090982           0.002755   
8b668d93 -28.114054       1.0          0.091786           0.004020   
1a1448e1 -28.077969       1.0          0.091507           0.002960   
39b8189a -28.058559       1.0          0.091538           0.002974   
996504df -27.999535       1.0          0.090843           0.003099   
d532aa5d -27.950037       1.0          0.088303           0.006702   
381b5ce4 -27.905373       1.0          0.067527           0.094744   
cb22fad4 -27.878670       1.0          0.078666           0.013706   
aa49ea84 -27.869978 

c:\Users\Timo\anaconda3\envs\pytorch\lib\site-packages\ray\tune\analysis\experiment_analysis.py:280: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


In [11]:
with open("../results/raytune_oja_ctx_blocked_8episodes.pkl", "rb") as f:
    df = pickle.load(f)

df.iloc[0]

mean_loss              -28.530746
mean_acc                 1.000000
config.lrate_sgd         0.092071
config.lrate_hebb        0.003988
config.ctx_scaling       3.000000
config.seed           9531.000000
Name: 16dfc485, dtype: float64

In [20]:
# verify results 
with open("../results/raytune_oja_ctx_blocked_8episodes.pkl", "rb") as f:
    df = pickle.load(f)
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]

# override defaults 
args.n_episodes = 8
args.lrate_hebb = df.iloc[0]["config.lrate_hebb"]
args.lrate_sgd = df.iloc[0]["config.lrate_sgd"]
args.ctx_scaling = df.iloc[0]["config.ctx_scaling"]
args.ctx_avg = False
np.random.seed(int(df.iloc[0]["config.seed"]))
random.seed(int(df.iloc[0]["config.seed"]))
torch.manual_seed(int(df.iloc[0]["config.seed"]))


# create dataset 
dataset = make_blobs_dataset(args)

# instantiate logger, model and optimiser:
logger = MetricLogger(save_dir)
model = Nnet(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)


# train model
train_on_blobs(args, model, optimiser, dataset, logger)

print(f"config: lrate_sgd: {args.lrate_sgd:.4f}, lrate_hebb: {args.lrate_hebb:.4f}, context offset: {args.ctx_scaling}")
print(f"terminal accuracy: {logger.acc_total[-1]:.2f}, loss: {logger.losses_total[-1]:.2f}")

step 0, loss: task a -0.3293, task b -0.0545 | acc: task a 0.5000, task b 0.5500
... n_a: 8 n_b: 8
step 50, loss: task a -9.5941, task b -0.2827 | acc: task a 1.0000, task b 0.5000
... n_a: 12 n_b: 0
step 100, loss: task a -13.5838, task b -0.0747 | acc: task a 1.0000, task b 0.5000
... n_a: 21 n_b: 0
step 150, loss: task a -14.3693, task b -0.0041 | acc: task a 1.0000, task b 0.5000
... n_a: 32 n_b: 1
step 200, loss: task a -14.6084, task b -0.0028 | acc: task a 1.0000, task b 0.5000
... n_a: 37 n_b: 2
step 250, loss: task a -14.3022, task b -5.6553 | acc: task a 1.0000, task b 0.9000
... n_a: 14 n_b: 1
step 300, loss: task a -14.1123, task b -12.3726 | acc: task a 1.0000, task b 1.0000
... n_a: 16 n_b: 14
step 350, loss: task a -14.3929, task b -14.1282 | acc: task a 1.0000, task b 1.0000
... n_a: 19 n_b: 18
done
config: lrate_sgd: 0.0921, lrate_hebb: 0.0040, context offset: 3.0
terminal accuracy: 1.00, loss: -28.52


### HPO: Interleaved trials

In [4]:
# HPO on blocked trials with oja_ctx
args = parser.parse_args(args=[])
args.n_episodes = 8
args.hpo_fixedseed = True
args.hpo_scheduler = "bohb"
args.hpo_searcher = "bohb"
args.training_schedule = "interleaved"
# dict(sorted(vars(args).items(),key=lambda k: k[0]))
args.ctx_avg = False
# init tuner
tuner = HPOTuner(args, time_budget=60*15, metric="loss")

tuner.tune(n_samples=500)

df = tuner.results
df = df[["mean_loss", "mean_acc", "config.lrate_sgd","config.lrate_hebb", "config.ctx_scaling","config.seed","done"]]
df = df[df["done"]==True]
df = df.drop(columns=["done"])
df = df.dropna()
df = df.sort_values("mean_loss",ascending=True)

df.reset_index()
print(df.head(15))

print(tuner.best_cfg)

with open("../results/raytune_oja_ctx_interleaved_8episodes.pkl", "wb") as f:
    pickle.dump(df, f)

2022-06-23 14:32:01,270	INFO tune.py:636 -- Total run time: 903.16 seconds (900.28 seconds for the tuning loop).


          mean_loss  mean_acc  config.lrate_sgd  config.lrate_hebb  \
trial_id                                                             
d46282a5 -29.282215       1.0          0.087100           0.005814   
e95cda9f -29.152634       1.0          0.083062           0.006928   
96431736 -28.942841       1.0          0.088884           0.016040   
67316b60 -28.915279       1.0          0.089539           0.020199   
63c268f6 -28.843061       1.0          0.089837           0.016160   
551533f5 -28.838718       1.0          0.094707           0.054363   
67180152 -28.759750       1.0          0.087407           0.018329   
a3faebae -28.733679       1.0          0.087999           0.013216   
97e88225 -28.678537       1.0          0.090390           0.023709   
a6beafdd -28.586058       1.0          0.085326           0.004948   
9911a805 -28.576925       1.0          0.086460           0.003645   
c3f444ad -28.526861       1.0          0.086867           0.003767   
67242e0d -28.523685 

In [7]:
# verify results 

# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]

# override defaults 
args.n_episodes = 8
args.lrate_hebb = df.iloc[0]["config.lrate_hebb"]
args.lrate_sgd = df.iloc[0]["config.lrate_sgd"]
args.ctx_scaling = df.iloc[0]["config.ctx_scaling"]
args.ctx_avg = False
args.training_schedule = "interleaved"
np.random.seed(int(df.iloc[0]["config.seed"]))
random.seed(int(df.iloc[0]["config.seed"]))
torch.manual_seed(int(df.iloc[0]["config.seed"]))



# create dataset 
dataset = make_blobs_dataset(args)

# instantiate logger, model and optimiser:
logger = MetricLogger(save_dir)
model = Nnet(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)


# train model
train_on_blobs(args, model, optimiser, dataset, logger)

print(f"config: lrate_sgd: {args.lrate_sgd:.4f}, lrate_hebb: {args.lrate_hebb:.4f}, context offset: {args.ctx_scaling}")
print(f"terminal accuracy: {logger.acc_total[-1]:.2f}, loss: {logger.losses_total[-1]:.2f}")

step 0, loss: task a -0.4063, task b -0.2303 | acc: task a 0.6500, task b 0.5000
... n_a: 7 n_b: 7
step 50, loss: task a -3.7139, task b -2.7193 | acc: task a 0.8500, task b 0.8000
... n_a: 1 n_b: 3
step 100, loss: task a -7.7052, task b -5.1555 | acc: task a 0.9500, task b 0.8000
... n_a: 3 n_b: 12
step 150, loss: task a -11.1321, task b -4.0942 | acc: task a 1.0000, task b 0.5000
... n_a: 7 n_b: 12
step 200, loss: task a -13.0944, task b -5.3615 | acc: task a 1.0000, task b 0.5500
... n_a: 10 n_b: 11
step 250, loss: task a -10.1326, task b -14.1101 | acc: task a 0.8000, task b 1.0000
... n_a: 15 n_b: 11
step 300, loss: task a -14.4625, task b -13.2745 | acc: task a 1.0000, task b 1.0000
... n_a: 17 n_b: 14
step 350, loss: task a -14.7191, task b -14.5631 | acc: task a 1.0000, task b 1.0000
... n_a: 17 n_b: 14
done
config: lrate_sgd: 0.0871, lrate_hebb: 0.0058, context offset: 3
terminal accuracy: 1.00, loss: -29.28
